### Установка и импорт неообходимых модулей

In [4]:
!pip install pandas numpy tqdm

In [6]:
import pandas
from tqdm import tqdm
import numpy as np

In [8]:
## Загрузим исходные данные и выведим премер строки в них

In [10]:
df = pandas.read_excel('./dataset.xlsx', header=None)
df.sample(1)

0   \
128  Эстетическая косметология (Юниоры)   

                                                    1   \
128  «Эстетическая косметология». Именование работа...   

                                                    2   3   \
128  Организация рабочего места ,нормативные документы  13   

                                                    4             5   6   \
128  Специалист должен знать и понимать: • законы, ...  Коммуникации  12   

                                                    7   \
128  Специалист должен знать и понимать: • професси...   

                                    8   9   ...   22   23  24   25   26  27  \
128  Анализ, планирование и менеджмент  15  ...  NaN  NaN NaN  NaN  NaN NaN   

      28   29   30   31  
128  NaN  NaN  NaN  NaN  

[1 rows x 32 columns]

#### Данные находятся в неудобном для обычной таблицы формате, для структуризации их необходимо загрузить в словарь по компетецниям и в каждой компетенции держать список неообходимых навыков

In [12]:
comps= {}
for i, row in df.iterrows():
    skills = []
    skill = {'competetion_description': row[1]}
    for i, item in row.iteritems():
        if i > 1 and not pandas.isnull(item):
            if i % 3 == 2:
                skill['knowledge'] = item
            if i % 3 == 0:
                skill['weight'] = item
            if i % 3 == 1:
                skill['skill_description'] = item
                if i>3:
                    skills.append(skill.copy())
            
            
    comps[row[0]] = skills

#### Выведем пример компетенции и неообходимого навыка из словаря для проверки

In [14]:
comps['Преподавание в младших классах(юниоры)'][1]

{'competetion_description': 'Высококвалифицированные специалисты в сфере начального общего\nобразования владеют широким спектром современных технологий,\nпозволяющих эффективно решать задачи, связанных с обучением и\nвоспитанием обучающихся младших классов. Обучение и воспитание\nпредполагает создание условий для всестороннего развития личности ребенка.\nПреподаватель в младших классах должен уметь создать такие условия в\nразных видах деятельности: урочной и внеурочной. Полноценное создание\nусловий обеспечивается компетентностью преподавателя начальных классов в\nвопросах взаимодействия с участниками образовательных отношений\n(проведение мастер-классов для коллег, организация родительского собрания),\nв вопросах постоянного повышения квалификации и самообразования (ведение\nперсонального сайта в информационно-телекоммуникационной сети интернет,\nоперативное решение ситуативных педагогических задач).\nПреподавателю в младших классах необходим высокий уровень\nспециальных знаний и нав

### Дополним словарь компетенций данными из приложеных технический описаний

In [16]:
comp_description = "Специалист по сметному делу – это рабоотник оргганизации, который отвечает за разработку,проверрку и анализ сметной доокументации любой сложности на основаниии сходнных данных, в раммках техннического заданияи действующей нормативной докумментации."

comp = [{
    'competetion_description': comp_description,
    'knowledge': 'Получение, анализ и подготовка исходных данных для определения сметной стоимости строительства и разработки сметной документации',
    'weight': 30,
    'skill_description': "Специалист должен знать и понимать: * Нормативные правовые акты Российской Федерации, руководящие документы, методические и справочные материалы, относящиеся к сфере регулирования ценообразования и сметного нормирования в области I радос J роительной деятельности; * Система правовых и нормативных требований к оформлению, комплектации и представлению различных видов градостроительной документации, составу разделов проектной документации и требований к их содержанию; * Общие правила выполнения п оформления текстовых и графических материалов, входящих в состав проектной и рабочей документации; * Современные средства автоматизации и технологии осуществления работ, автоматизированные информационные и телекоммуникационные системы. Специалист должен уметь: * Оценивать состав и содержание проектной документации и других исходных данных, необходимых для определения сметной стоимости строительства, в соответствии с установленными требованиями; * Использовать проектную, нормативную правовую, нормативно-техническую документацию и справочную информацию для получения сведений, необходимых для определения сметной стоимости строительства; * Использовать современные автоматизированные информационные и телекоммуникационные системы, для получения и обработки исходных данных, необходимых для определения сметной стоимости строительства".replace('*', '•'),
}]
comps['Специалист по сметному делу'] = comp

#### Сохраним словарь что использования в будуюшем

In [18]:
import pickle
with open('comp_dict.pkl', 'wb') as file:
    pickle.dump(comps, file)

## Переведем в данные более удобный вид
#### Удобным форматом будет таблица с несколькими индексами где первостепенный индекс - названия компетенции, второстепенный - вес навыка, это позволяет применять фильтры по названию компетенции и фильтровать навыки по весу как внути компетенции так и по всем компетенциям сразу. В значениях будут находится названия навыка, его описание и описание компетенции.

In [20]:
inds = []
for key, val in comps.items():
    for skilla in val:
        if 'weight' in skilla.keys():
            inds.append((key, skilla['weight']))

In [22]:
multi_index = pandas.MultiIndex.from_tuples(inds, names=['name', 'weight'])
mi_df = pandas.DataFrame(index=multi_index, columns= ['knowledge', 'skill_description', 'competetion_description'])
mi_df

knowledge  \
name                                               weight             
Разработка компьютерных игр и мультимедийных пр... 13.0         NaN   
                                                   27.0         NaN   
                                                   30.0         NaN   
                                                   10.0         NaN   
                                                   15.0         NaN   
...                                                             ...   
Ювелирное дело                                     10.0         NaN   
                                                   20.0         NaN   
                                                   20.0         NaN   
                                                   15.0         NaN   
Специалист по сметному делу                        30.0         NaN   

                                                          skill_description  \
name                                               weight                     
Разработка компьютерных игр и мультимедийных пр... 13.0                 NaN   
                                                   27.0                 NaN   
                                                   30.0                 NaN   
                                                   10.0                 NaN   
                                                   15.0                 NaN   
...                                                                     ...   
Ювелирное дело                                     10.0                 NaN   
                                                   20.0                 NaN   
                                                   20.0                 NaN   
                                                   15.0                 NaN   
Специалист по сметному делу                        30.0                 NaN   

                                                          competetion_description  
name                                               weight                          
Разработка компьютерных игр и мультимедийных пр... 13.0                       NaN  
                                                   27.0                       NaN  
                                                   30.0                       NaN  
                                                   10.0                       NaN  
                                                   15.0                       NaN  
...                                                                           ...  
Ювелирное дело                                     10.0                       NaN  
                                                   20.0                       NaN  
                                                   20.0                       NaN  
                                                   15.0                       NaN  
Специалист по сметному делу                        30.0                       NaN  

[820 rows x 3 columns]

In [24]:
for key, val in comps.items():
    for skill in val:
        if 'weight' in skill.keys():
            mi_df.loc[(key, skill['weight']),:] = (skill['knowledge'], skill['skill_description'], skill['competetion_description'])
        #mi_df[key, skill['weight']] = [skill['knowledge'], skill['description']]

#### Проверим что во время преобразования в талблицу небыло потеряно значений, а также что в ней находятся названия компетенций их описания, и описание нееобходимых знаний сохраними промежуточный результат в формат csv

In [26]:
print(mi_df.index)
print(mi_df.columns)
print(len(mi_df.dropna()) == len(mi_df))
mi_df

MultiIndex([('Разработка компьютерных игр и мультимедийных приложений',   13),
            ('Разработка компьютерных игр и мультимедийных приложений',   27),
            ('Разработка компьютерных игр и мультимедийных приложений',   30),
            ('Разработка компьютерных игр и мультимедийных приложений',   10),
            ('Разработка компьютерных игр и мультимедийных приложений',   15),
            ('Разработка компьютерных игр и мультимедийных приложений',  5.0),
            (                'Разработка мобильных приложений(юниоры)',  5.0),
            (                'Разработка мобильных приложений(юниоры)',  5.0),
            (                'Разработка мобильных приложений(юниоры)',   30),
            (                'Разработка мобильных приложений(юниоры)',   50),
            ...
            (                     'Эстетическая косметология (Юниоры)',   15),
            (                     'Эстетическая косметология (Юниоры)',   25),
            (                       

knowledge  \
name                                               weight                                                      
Разработка компьютерных игр и мультимедийных пр... 13.0                     Организация и управление работой   
                                                   27.0                         Программирование на языке С#   
                                                   30.0                                             Unity 3D   
                                                   10.0            Оптимизация компьютерных игр и приложений   
                                                   15.0                             Анализ и проектирование    
...                                                                                                      ...   
Ювелирное дело                                     10.0    Подготовка драгоценных сплавов для изготовлени...   
                                                   20.0    Изготовление сложных элементов и ювелирных изд...   
                                                   20.0    Изготовление сложных элементов и ювелирных изд...   
                                                   15.0                                Обработка поверхности   
Специалист по сметному делу                        30.0    Получение, анализ и подготовка исходных данных...   

                                                                                           skill_description  \
name                                               weight                                                      
Разработка компьютерных игр и мультимедийных пр... 13.0    Специалист должен знать и понимать: • Принципы...   
                                                   27.0    Специалист должен знать и понимать: • Базовые ...   
                                                   30.0    Специалист должен знать и понимать: • Основные...   
                                                   10.0    Специалист должен знать и понимать: • Особенно...   
                                                   15.0    Специалист должен знать и понимать: • Важность...   
...                                                                                                      ...   
Ювелирное дело                                     10.0    Специалист должен знать и понимать: • Свойства...   
                                                   20.0    Специалист должен знать и понимать: • Способы ...   
                                                   20.0    Специалист должен знать и понимать: • Способы ...   
                                                   15.0    Специалист должен знать и понимать: • Специали...   
Специалист по сметному делу                        30.0    Специалист должен знать и понимать: • Норматив...   

                                                                                     competetion_description  
name                                               weight                                                     
Разработка компьютерных игр и мультимедийных пр... 13.0    Разработка компьютерных игр и мультимедийных п...  
                                                   27.0    Разработка компьютерных игр и мультимедийных п...  
                                                   30.0    Разработка компьютерных игр и мультимедийных п...  
                                                   10.0    Разработка компьютерных игр и мультимедийных п...  
                                                   15.0    Разработка компьютерных игр и мультимедийных п...  
...                                                                                                      ...  
Ювелирное дело                                     10.0    Профессия ювелира, прежде всего, связана с руч...  
                                                   20.0    Профессия ювелира, прежде всего, связана с руч...  
                                                   20.0    Профессия ювелир

In [28]:
mi_df.to_csv('mi_dataset.csv')

## Выделение словарей аттрибутов
### Выделим необходимые навыки из списков и поместим в отдельный столбик они имеют наиболшее значение в кластеризации

In [30]:
def extract_skills(text):
    text = str(text)
    splited = text.split('•')
    skills = []
    for skill in splited[1:]:
        if not skill.startswith('Участнику') and not skill.startswith('Специалист'):
            skills.append(skill)
    return ':'.join(skills)

tqdm.pandas()
mi_df['skills'] = mi_df['skill_description'].progress_apply(extract_skills)

100%|██████████| 820/820 [00:00<00:00, 82050.94it/s]


### Выделим ключевые слова в описании навыков и компетенцияй так как они имеют большое значение для кластеризации

In [32]:
!pip install rutermextract

In [34]:
from rutermextract import TermExtractor
term_extractor = TermExtractor()

In [36]:
def extract_terms(text):
    text = str(text)
    terms = term_extractor(text)
    norm_terms_list = [term.normalized for term in terms]
    return ':'.join(norm_terms_list)

In [38]:
tqdm.pandas()
mi_df['skill_description'] = mi_df['skill_description'].progress_apply(extract_terms)

100%|██████████| 820/820 [00:12<00:00, 66.34it/s]


In [43]:
tqdm.pandas()
mi_df['competetion_description'] = mi_df['competetion_description'].progress_apply(extract_terms)

3%|▎         | 26/820 [00:04<02:09,  6.13it/s]


KeyboardInterrupt: 

# Сохранним данные в processed dataset.csv

In [42]:
mi_df.to_csv('processed_dataset.csv')